<a href="https://colab.research.google.com/github/Guylord/APHRC_DSE_Inspire_Hackathon_2024/blob/main/APHRC_DSE_Insprire_Hackathon_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install recordlinkage

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.9/926.9 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 58.9 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import recordlinkage as rl

In [4]:
hdss_df = pd.read_csv('/content/drive/MyDrive/APHRC-DSE Inspire Hackathon 2024/synthetic_hdss_v3.csv')
facility_df = pd.read_csv('/content/drive/MyDrive/APHRC-DSE Inspire Hackathon 2024/synthetic_facility_v3.csv')

print('HDSS INFO')
hdss_df.info()
print('\n\nFACILITY INFO')
facility_df.info()

HDSS INFO
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4115 entries, 0 to 4114
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   recnr       4115 non-null   int64  
 1   firstname   4115 non-null   object 
 2   lastname    4031 non-null   object 
 3   petname     2894 non-null   object 
 4   dob         4115 non-null   object 
 5   sex         4115 non-null   int64  
 6   nationalid  0 non-null      float64
 7   hdssid      4115 non-null   object 
 8   hdsshhid    4115 non-null   object 
dtypes: float64(1), int64(2), object(6)
memory usage: 289.5+ KB


FACILITY INFO
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2902 entries, 0 to 2901
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   recnr       2902 non-null   int64 
 1   firstname   2902 non-null   object
 2   lastname    2835 non-null   object
 3   petname     2511 non-null   object
 4   do

# *FACILITY dataset*

In [5]:
facility_df.head()

,recnr,firstname,lastname,petname,dob,sex,nationalid,patientid,visitdate
0,2,Fatuma,NaN,Zaina,24-08-2017 00:00,2,N_ID_5000,2069,10-09-2018
1,3,Gloria,Rashida,NaN,11-07-1993 00:00,2,N_ID_11861,2079,14-12-2022
2,4,Ali,Hakram,Igomu,17-05-2014 00:00,1,N_ID_11864,2080,09-06-2023
3,5,Nakalema,NaN,Nkwanga,27-02-2026 00:00,2,N_ID_11867,2081,07-02-2019
4,6,Asuman,Sempa,Aguti,02-03-2002 00:00,1,N_ID_11870,2082,18-08-2020


In [6]:
from recordlinkage.preprocessing import clean

for col in facility_df.select_dtypes('object').columns:
  clean(facility_df[col], replace_by_none='[^ \\-\\_A-Za-z0-9]+', remove_brackets=True)

In [ ]:
facility_df['dob'] = pd.to_datetime(facility_df['dob'])
facility_df['visitdate'] = pd.to_datetime(facility_df['visitdate'])

<ipython-input-10-9b6fa05c9244>:2: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  facility_df['visitdate'] = pd.to_datetime(facility_df['visitdate'])


In [ ]:
facility_df.isna().sum()

recnr           0
firstname       0
lastname       67
petname       391
dob             0
sex             0
nationalid      0
patientid       0
visitdate       0
dtype: int64

In [ ]:
facility_df.duplicated().sum()

0

In [ ]:
facility_df.duplicated(subset=['firstname', 'lastname']).sum()

641

In [ ]:
facility_df[facility_df.duplicated(subset=['firstname', 'lastname'], keep=False)].sort_values(['firstname', 'lastname']).head(10)

,recnr,firstname,lastname,petname,dob,sex,nationalid,patientid,visitdate
235,237,Abdallah,Richard,Afa,2021-12-09,1,N_ID_5666,2307,2022-08-20
1460,1462,Abdallah,Richard,Afa,2021-08-23,1,N_ID_9344,3531,2022-03-21
2581,2583,Abubakali,Lunkumu,Jjunju,1934-07-08,1,N_ID_12749,4651,2022-12-21
2684,2686,Abubakali,Lunkumu,Jjunju,1934-07-08,1,N_ID_13055,4754,2019-01-06
371,373,Abubakali,Luwazo,Katabazi,1993-10-27,1,N_ID_6080,2443,2022-06-21
997,999,Abubakali,Luwazo,Nalubega,1993-07-11,1,N_ID_7958,3068,2018-11-19
573,575,Abuneli,Lawrence,Asega,1959-11-08,1,N_ID_6689,2645,2022-03-07
1160,1162,Abuneli,Lawrence,Asega,1959-11-08,1,N_ID_8444,3231,2019-09-20
2531,2533,Aidah,Fatuma,Aya,2011-12-13,2,N_ID_12599,4601,2020-06-23
2588,2590,Aidah,Fatuma,Aya,2011-12-13,2,N_ID_12770,4658,2020-04-19


In [ ]:
facility_df.duplicated(subset=['firstname', 'lastname', 'dob']).sum()

294

In [ ]:
facility_df[facility_df.duplicated(subset=['firstname', 'lastname', 'dob'], keep=False)].sort_values('dob').head(10)

,recnr,firstname,lastname,petname,dob,sex,nationalid,patientid,visitdate
1517,1519,Rayan,Sowobi,Nalubega,1930-09-23,1,N_ID_9515,3588,2019-05-25
1360,1362,Rayan,Sowobi,Nalubega,1930-09-23,1,N_ID_9044,3431,2018-04-08
1432,1434,Hassan,Isa,Deo,1931-05-24,1,N_ID_9260,3503,2022-09-15
1745,1747,Hassan,Isa,Deo,1931-05-24,1,N_ID_10202,3816,2021-02-22
733,735,Derick,Falidi,Akili,1931-07-06,1,N_ID_7175,2805,2021-04-02
545,547,Derick,Falidi,Akili,1931-07-06,1,N_ID_6605,2617,2022-12-11
1595,1597,Nakidodo,Nakagolo,Nambi,1932-06-12,2,N_ID_9749,3666,2021-06-01
644,646,Nakidodo,Nakagolo,Nambi,1932-06-12,2,N_ID_6902,2716,2021-02-26
677,679,Sawuya,Namukose,Amutu,1932-11-30,2,N_ID_7001,2749,2020-09-02
155,157,Sawuya,Namukose,Amutu,1932-11-30,2,N_ID_5426,2227,2019-10-25


In [ ]:
facility_df.duplicated(subset='nationalid', keep=False).sum()

0



* It's evident that some records have the same entries for 'firstname,' 'lastname,' 'petname,' and 'dob,' implying that they refer to the same entity but with different 'nationalid' entries during various visits to the facility. This suggests that individuals visiting the facility did not retain a single 'nationalid,' rendering 'nationalid' unusable as a unique identifier

* Individuals have different 'patientid' entries at each visit to the health facility






# *HDSS dataset*

In [ ]:
hdss_df.head()

,recnr,firstname,lastname,petname,dob,sex,nationalid,hdssid,hdsshhid
0,1,Zaina,Hanifa,Ula,22-09-1930 00:00,2,NaN,I20001,HH100001
1,2,Godfrey,Maganda,Mukama,15-07-1934 00:00,1,NaN,I20002,HH100002
2,3,Kasim,Ngobi,Galabuzi,03-03-1983 00:00,1,NaN,I20003,HH100003
3,4,Esther,NaN,Inara,30-07-1968 00:00,2,NaN,I20004,HH100004
4,5,Sumaya,Swabula,NaN,13-12-1930 00:00,2,NaN,I20005,HH100005


In [ ]:
for col in hdss_df.select_dtypes('object').columns:
  clean(hdss_df[col], replace_by_none='[^ \\-\\_A-Za-z0-9]+', remove_brackets=True)

In [ ]:
hdss_df['dob'] = pd.to_datetime(hdss_df['dob'])

In [ ]:
hdss_df.isna().sum()

recnr            0
firstname        0
lastname        84
petname       1221
dob              0
sex              0
nationalid    4115
hdssid           0
hdsshhid         0
dtype: int64

In [ ]:
hdss_df.duplicated().sum()

0

In [ ]:
hdss_df.duplicated(subset=['firstname', 'lastname', 'petname']).sum()

16

In [ ]:
mask = hdss_df.duplicated(subset=['firstname', 'lastname', 'petname'], keep=False)
hdss_df[mask].sort_values('firstname')

,recnr,firstname,lastname,petname,dob,sex,nationalid,hdssid,hdsshhid
3770,3771,Jalia,Naigaga,NaN,2021-01-06,2,NaN,I23771,HH103771
2800,2801,Jalia,Naigaga,NaN,1959-08-04,2,NaN,I22801,HH102801
1072,1073,Janet,Naigaga,NaN,2006-03-03,2,NaN,I21073,HH101073
1133,1134,Janet,Naigaga,NaN,1994-06-10,2,NaN,I21134,HH101134
1175,1176,Kagoya,Shaluwa,NaN,1979-10-23,2,NaN,I21176,HH101176
3908,3909,Kagoya,Shaluwa,NaN,1988-06-02,2,NaN,I23909,HH103909
557,558,Kaudha,Nakaziba,NaN,1930-04-09,2,NaN,I20558,HH100558
2450,2451,Kaudha,Nakaziba,NaN,1933-09-29,2,NaN,I22451,HH102451
2157,2158,Mutesi,Nangobi,NaN,2015-02-07,2,NaN,I22158,HH102158
258,259,Mutesi,Nangobi,NaN,1954-03-23,2,NaN,I20259,HH100259


In [ ]:
hdss_df.duplicated(subset=['firstname', 'lastname', 'dob'], keep=False).sum()

0

In [ ]:
hdss_df.duplicated(subset=['firstname', 'lastname'], keep=False).sum()

167

In [ ]:
hdss_df[hdss_df.duplicated(subset=['firstname', 'lastname'], keep=False)].sort_values('firstname').head(10)

,recnr,firstname,lastname,petname,dob,sex,nationalid,hdssid,hdsshhid
2470,2471,Aisa,NaN,Zama,1991-01-19,2,NaN,I22471,HH102471
1712,1713,Aisa,NaN,NaN,2004-09-20,2,NaN,I21713,HH101713
2060,2061,Amina,Nabirye,NaN,1994-05-29,2,NaN,I22061,HH102061
2994,2995,Amina,Nabirye,Sira,1984-06-06,2,NaN,I22995,HH102995
44,45,Babirye,Mutesi,NaN,1949-07-18,2,NaN,I20045,HH100045
1721,1722,Babirye,Babirye,NaN,1963-04-17,2,NaN,I21722,HH101722
2979,2980,Babirye,Babirye,Pia,1985-07-09,2,NaN,I22980,HH102980
3981,3982,Babirye,Mutesi,Nyala,2022-02-09,2,NaN,I23982,HH103982
3647,3648,Babra,Shakira,Kiara,1948-07-28,2,NaN,I23648,HH103648
2138,2139,Babra,Shakira,Anaya,1959-12-29,2,NaN,I22139,HH102139


In [ ]:
hdss_df.duplicated(subset='hdssid').sum()

0

In [ ]:
hdss_df.duplicated(subset='hdsshhid').sum()

0

In [ ]:
indexer = rl.Index()
indexer.block(left_on='dob', right_on='dob')
pairs = indexer.index(facility_df, hdss_df)

In [ ]:
compare = rl.Compare()

compare.string('firstname', 'firstname', method='jarowinkler', label='firstname')
compare.string('petname', 'petname', method='jarowinkler', threshold=0.85, label='petname')
compare.string('lastname', 'lastname', method='jarowinkler', label='lastname')
compare.date('dob', 'dob', label='dob')
compare.exact('sex', 'sex', label='sex')


potential_matches=compare.compute(pairs, facility_df, hdss_df)
potential_matches

,,firstname,petname,lastname,dob,sex
0,2740,1.0,0.0,0.0,1,1
1,2612,1.0,0.0,1.0,1,1
3,1178,1.0,0.0,0.0,1,1
6,2901,1.0,1.0,1.0,1,1
7,2950,1.0,1.0,1.0,1,1
...,...,...,...,...,...,...
2884,2126,1.0,0.0,0.0,1,1
2892,3320,1.0,1.0,1.0,1,1
2893,691,1.0,1.0,1.0,1,1
2898,3446,1.0,0.0,1.0,1,1


In [ ]:
#lr=recordlinkage.LogisticRegressionClassifier()
#nb=recordlinkage.NaiveBayesClassifier()
#ecm=recordlinkage.ECMClassifier()
kmeans=rl.KMeansClassifier()
kmeans.fit(potential_matches)
matched_indices = kmeans.predict(potential_matches)

In [ ]:
hdss_facility = pd.concat([facility_df.loc[matched_indices.get_level_values(0)], hdss_df.loc[matched_indices.get_level_values(1)]])
hdss_facility.sort_values(['firstname', 'lastname']).head(30)

,recnr,firstname,lastname,petname,dob,sex,nationalid,patientid,visitdate,hdssid,hdsshhid
1207,1209,Ababu,Crispus,Lwasa,2006-04-23,1,N_ID_8585,3278.0,2020-01-15,NaN,NaN
3481,3482,Ababu,Crispus,Lwasa,2006-04-23,1,NaN,NaN,NaT,I23482,HH103482
678,680,Abatuka,Nahiya,Fanda,1985-02-25,2,N_ID_7004,2750.0,2018-08-29,NaN,NaN
3245,3246,Abatuka,Nahiya,Fanda,1985-02-25,2,NaN,NaN,NaT,I23246,HH103246
235,237,Abdallah,Richard,Afa,2021-12-09,1,N_ID_5666,2307.0,2022-08-20,NaN,NaN
2312,2313,Abdallah,Richard,Afa,2021-12-09,1,NaN,NaN,NaT,I22313,HH102313
1978,1980,Abdu,Muwunba,Mwijukye,1996-04-19,1,N_ID_10901,4048.0,2019-06-01,NaN,NaN
3849,3850,Abdu,Muwunba,Mwijukye,1996-04-19,1,NaN,NaN,NaT,I23850,HH103850
2018,2020,Abdu,Muzeyi,Muhammed,1969-03-20,1,N_ID_11021,4088.0,2021-09-01,NaN,NaN
76,77,Abdu,Muzeyi,Muhammed,1969-03-20,1,NaN,NaN,NaT,I20077,HH100077


In [ ]:
hdss_facility.shape

(1744, 11)

In [ ]:
hdss_.duplicated().sum()

0

In [ ]:
facility_hdss.duplicated(subset=['fullname', 'petname', 'dob', 'sex']).sum()

879

In [ ]:
facility_hdss.drop_duplicates(subset=['fullname', 'petname', 'dob', 'sex'], keep='first')
facility_hdss

,firstname,lastname,petname,dob,sex,nationalid,hdssid,hdsshhid,fullname,patientid,visitdate
0,Zaina,Hanifa,Ula,22-09-1930 00:00,2,NaN,I20001,HH100001,Zaina Hanifa,NaN,NaN
1,Godfrey,Maganda,Mukama,15-07-1934 00:00,1,NaN,I20002,HH100002,Godfrey Maganda,NaN,NaN
2,Kasim,Ngobi,Galabuzi,03-03-1983 00:00,1,NaN,I20003,HH100003,Kasim Ngobi,NaN,NaN
3,Esther,NaN,Inara,30-07-1968 00:00,2,NaN,I20004,HH100004,Esther,NaN,NaN
4,Sumaya,Swabula,NaN,13-12-1930 00:00,2,NaN,I20005,HH100005,Sumaya Swabula,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2895,Swabiru,Aluvin,Najjuma,19-05-2014 00:00,1,N_ID_13670,NaN,NaN,Swabiru Aluvin,4959.0,16-10-2022
2897,Madina,Nakagolo,Nkima,28-07-1955 00:00,2,N_ID_13676,NaN,NaN,Madina Nakagolo,4961.0,17-08-2021
2898,Namulondo,Namugabwe,NaN,14-09-1933 00:00,2,N_ID_13679,NaN,NaN,Namulondo Namugabwe,4962.0,21-06-2019
2899,Ziriya,Kauma,Kato,11-11-1963 00:00,2,N_ID_13682,NaN,NaN,Ziriya Kauma,4963.0,27-12-2018


In [ ]:
facility_hdss.shape

(6395, 11)

In [ ]:
duplicates_mask = facility_hdss.duplicated(subset=['fullname', 'petname', 'dob', 'sex'])

# Filter the DataFrame to keep only the duplicate rows
duplicate_rows = facility_hdss[duplicates_mask]

# Display the result
print("Duplicate Rows:")
print(duplicate_rows)


Duplicate Rows:
      firstname   lastname    petname               dob  sex  nationalid  \
1        Gloria    Rashida        NaN  11-07-1993 00:00    2  N_ID_11861   
6     Namutamba     Sumaya       Bria  18-02-1997 00:00    2  N_ID_11876   
10       Nasiri    Kirunda      Ebitu  02-06-1959 00:00    1   N_ID_5015   
16       Hajara   Namusubo        NaN  02-12-1955 00:00    2   N_ID_5033   
29       Aminsi       Ibra       Jose  22-12-2000 00:00    1   N_ID_5072   
...         ...        ...        ...               ...  ...         ...   
2891     Safalu        NaN       Atek  07-04-2025 00:00    1  N_ID_13658   
2893     Ukasha  Gubayanga    Kulubya  24-12-1987 00:00    1  N_ID_13664   
2898  Namulondo  Namugabwe        NaN  14-09-1933 00:00    2  N_ID_13679   
2899     Ziriya      Kauma       Kato  11-11-1963 00:00    2  N_ID_13682   
2900      Faizo    Buyinza  Ssenyonjo  09-06-1956 00:00    1  N_ID_13685   

     hdssid hdsshhid             fullname  patientid   visitdate  
1   

In [ ]:
facility_hdss[facility_hdss['fullname'] == 'Gloria Rashida']

,firstname,lastname,petname,dob,sex,nationalid,hdssid,hdsshhid,fullname,patientid,visitdate
2612,Gloria,Rashida,NaN,11-07-1993 00:00,2,NaN,I22613,HH102613,Gloria Rashida,NaN,NaN
1,Gloria,Rashida,NaN,11-07-1993 00:00,2,N_ID_11861,NaN,NaN,Gloria Rashida,2079.0,14-12-2022
2651,Gloria,Rashida,Bugingo,21-06-1993 00:00,2,N_ID_12956,NaN,NaN,Gloria Rashida,4721.0,12-07-2022
